In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm

cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')
#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = pm.sort("df_jo", ['created_at','created_at_fe']).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
#df_jo_cp = df_jo.copy()
#df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()
pd.options.display.max_columns = None
#display(df_jo)

df = df_jo 
pm.format_to_dates(df, time_format='s') # 'min','s'
fields = ['id_cr','user_id','created_at','moderated_at','transfer_type','stat_cr' ,'amount','fee',
          'n_fees','n_backs','needs_m_check_recov', 'n_recovery', 'n_incidents', #'created_at_dow',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment','type','recovery_status' 
          # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          ]

In [32]:
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by=['user_id', 'created_at'])

# obtenemos la primera fila de cada 'user_id' (la más antigua)
primer_registro = df.groupby('user_id').first()
primer_registro['no_moderat'] = primer_registro['moderated_at'].isnull()
no_moderats = primer_registro.reset_index() #level='user_id')
no_moderats = primer_registro[primer_registro['no_moderat'] == True]
#resultat = no_moderats.reset_index()[['id_cr','id_fe','user_id','created_at','moderated_at','','no_moderat','transfer_type']]
resultat = no_moderats[no_moderats.transfer_type.isin(['regular','instant'])].reset_index()
#display(resultat.sort_values(['created_at']))
#display(resultat[resultat.user_id > 0 ][fields].sort_values(['created_at','created_at_fe']).reset_index(drop=True))
display(resultat[fields].sort_values('created_at').head(4))
#resultat.user_id.count()

df['moderada'] = df['moderated_at'].apply(lambda x: 1 if not pd.isnull(x) else 0)
cnt = df[df['moderada'] == 0]
print(cnt.moderada.count())


,id_cr,user_id,created_at,moderated_at,transfer_type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_incidents,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status
4917,3772,99001956,2020-06-17 21:46:47,NaT,regular,canceled,50.0,0.0,0,0,1,0,1,cr_regular,0,NaT,NaT,None,NaT,2020-06-26 22:00:00,9 days 00:13:12.303552,NaT,NaT,None,0,nice
4916,3808,99001953,2020-06-18 02:32:07,NaT,regular,canceled,100.0,0.0,0,0,1,0,1,cr_regular,0,NaT,NaT,None,NaT,2020-07-05 22:00:00,17 days 19:27:52.576049,NaT,NaT,None,0,nice
4918,3937,99002028,2020-06-18 13:25:27,NaT,regular,canceled,100.0,0.0,0,0,1,0,1,cr_regular,0,NaT,NaT,None,NaT,2020-07-03 22:00:00,15 days 08:34:32.739437,NaT,NaT,None,0,nice
4919,3980,99002037,2020-06-18 16:02:48,NaT,regular,canceled,50.0,0.0,0,0,1,0,1,cr_regular,0,NaT,NaT,None,NaT,2020-07-01 22:00:00,13 days 05:57:11.970462,NaT,NaT,None,0,nice


10335
